# Skforecast 0.14 migration guide

This document contains a set of instructions on how to update your code to work with Skforecast 0.14.

## Changes in the import of the library

Skforecast modules have been reorganized and renamed to make it more intuitive. The new import of the library is as follows:

<table>
<tr>
<td style="text-align: center;"><strong>skforecast &lt; 0.14</strong></td>
<td style="text-align: center;"><strong>skforecast 0.14</strong></td>
</tr>
<tr>
<td>

```python
from skforecast.ForecasterAutoreg import ForecasterAutoreg
```
</td>
<td>

```python
from skforecast.recursive import ForecasterRecursive
```
</td>
</tr>
<tr>
<td>

```python
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
```
</td>
<td>

Not exist any more, use `ForecasterRecursive`

</td>
</tr>
<tr>
<td>

```python
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
```
</td>
<td>

```python
from skforecast.direct import ForecasterDirect
```
</td>
</tr>
</table>

## Custom features (window features)

Forecasters `ForecasterAutoregCustom` and `ForecasterAutoregMultiSeriesCustom` have been removed. Now, you can use `ForecasterRecursive` and `ForecasterDirect` and include custom features using the 
`window_features` parameter. To learn more about how to include custom features, check the [documentation]().

<table>

<tr>
    <td style="text-align: center;"><strong>skforecast &lt; 0.14</strong></td>
    <td style="text-align: center;"><strong>skforecast 0.14</strong></td>
</tr>

<tr>
<td style="vertical-align: top;">

```python
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
import numpy as np

def create_predictors(y):
    """
    Create first 10 lags of a time series.
    Calculate moving average with window 20.
    Calculate the moving standard deviation with window 20.
    Calculate moving minimum and maximum with window 20.
    """
    lags = y[-1:-11:-1]
    mean = np.mean(y[-20:])
    std = np.std(y[-20:])
    min_val = np.min(y[-20:])
    max_val = np.max(y[-20:])

    predictors = np.hstack([lags, mean, std, min_val, max_val])

    return predictors

feature_names = [f"lag {i}" for i in range(1, 11)] + [
    "moving_avg_20",
    "moving_std_20",
    "moving_min_20",
    "moving_max_20",
]

forecaster = ForecasterAutoregCustom(
    regressor       = LGBMRegressor(random_state=123, verbose=-1),
    fun_predictors  = create_predictors,
    name_predictors = feature_names,
    window_size     = 20
)
```

</td>

<td style="vertical-align: top;">

```python
from skforecast.recursive import ForecasterRecursive
from skforecast.preprocessing import RollingFeatures

rolling = RollingFeatures(
    ststs        = ['mean', 'std', 'min', 'max'],
    window_sizes = [20, 20, 20, 20]
)

forecaster = ForecasterRecursive(
    regressor       = LGBMRegressor(random_state=123, verbose=-1),
    lags            = 10,
    window_features = rolling
)
```
</td>

</tr>

</table>

## Changes in the backtesting

The arguments `initial_train_size`, `window_size`, `differentiation`, `refit`, `fixed_train_size`, `gap`, `skip_folds`, `allow_incomplete_fold`, `return_all_indexes`, and `verbose` are no longer defined in the `backtesting_forecaster` function. Instead, an instance of `TimeSeriesFolds` should be created, and these arguments should be specified in the class constructor. This change not only allows the same folds to be reused across different methods, but also provides the ability to extract fold indexes.

<table>

<tr>
    <td style="text-align: center;"><strong>skforecast &lt; 0.14</strong></td>
    <td style="text-align: center;"><strong>skforecast 0.14</strong></td>
</tr>

<tr>
<td style="vertical-align: top;">

```python
from skforecast.model_selection import (
    backtesting_forecaster
)

backtesting_forecaster(
    forecaster            = forecaster,
    y                     = y,
    steps                 = 10,
    initial_train_size    = 100,
    metric                = 'mean_absolute_error',
    fixed_train_size      = True,
    gap                   = 0,
    skip_folds            = None,
    allow_incomplete_fold = True,
    refit                 = False,
    n_jobs                = 'auto',
    verbose               = False,
    show_progress         = True
)
```

</td>

<td style="vertical-align: top;">

```python
from skforecast.model_selection import (
    TimeSeriesFold,
    backtesting_forecaster
)

cv = TimeSeriesFold(
    steps                 = 10,
    initial_train_size    = 100,
    fixed_train_size      = True,
    gap                   = 0,
    skip_folds            = None,
    allow_incomplete_fold = True,
    refit                 = False,
)

backtesting_forecaster(
    forecaster            = forecaster,
    y                     = y,
    cv                    = cv,
    metric                = 'mean_absolute_error',
    n_jobs                = 'auto',
    verbose               = False,
    show_progress         = True
)
```
</td>

</tr>

</table>

## Changes in hyperparameters search

The arguments `initial_train_size`, `window_size`, `differentiation`, `refit`, `fixed_train_size`, `gap`, `skip_folds`, `allow_incomplete_fold`, `return_all_indexes`, and `verbose` are no longer defined in the `grid_search_forecaster`, `random_search_forecaster` and `bayesian_search_forecaster` functions. Instead, an instance of `TimeSeriesFolds` or `OneStepAheadFold` should be created, and these arguments should be specified in the class constructor.

<table>

<tr>
    <td style="text-align: center;"><strong>skforecast &lt; 0.14</strong></td>
    <td style="text-align: center;"><strong>skforecast 0.14</strong></td>
</tr>

<tr>
<td style="vertical-align: top;">

```python
from skforecast.model_selection import (
    grid_search_forecaster
)

grid_search_forecaster(
    forecaster            = forecaster,
    y                     = data,
    param_grid            = param_grid,
    lags_grid             = lags_grid,
    steps                 = 10,
    refit                 = False,
    metric                = 'mean_squared_error',
    initial_train_size    = 100,
    fixed_train_size      = False,
    allow_incomplete_fold = True,
    return_best           = True,
    n_jobs                = 'auto',
    verbose               = False,
    show_progress         = True
)
```

</td>

<td style="vertical-align: top;">

```python
from skforecast.model_selection import (
    grid_search_forecaster,
    TimeSeriesFold
)

cv = TimeSeriesFold(
    steps                 = 10,
    initial_train_size    = 100,
    fixed_train_size      = False,
    gap                   = 0,
    skip_folds            = None,
    allow_incomplete_fold = True,
    refit                 = False,
)

grid_search_forecaster(
    forecaster         = forecaster,
    y                  = data,
    cv                 = cv,
    param_grid         = param_grid,
    lags_grid          = lags_grid,
    metric             = 'mean_squared_error',
    return_best        = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)
```
</td>

</tr>

</table>

<table>
    <tr>
        <td style="text-align: center;"><strong>skforecast &lt; 0.14</strong></td>
        <td style="text-align: center;"><strong>skforecast 0.14</strong></td>
    </tr>
    <tr>
        <td style="vertical-align: top;">

<pre><code class="python">
from skforecast.model_selection import backtesting_forecaster

backtesting_forecaster(
    forecaster            = forecaster,
    y                     = y,
    steps                 = 10,
    initial_train_size    = 100,
    metric                = 'mean_absolute_error',
    fixed_train_size      = True,
    gap                   = 0,
    skip_folds            = None,
    allow_incomplete_fold = True,
    refit                 = False,
    n_jobs                = 'auto',
    verbose               = False,
    show_progress         = True
)
</code></pre>

</td>
<td style="vertical-align: top;">

<pre><code class="python">
from skforecast.model_selection import (
    backtesting_forecaster,
    TimeSeriesFold
)

cv = TimeSeriesFold(
    steps                 = 10,
    initial_train_size    = 100,
    fixed_train_size      = True,
    gap                   = 0,
    skip_folds            = None,
    allow_incomplete_fold = True,
    refit                 = False,
)

backtesting_forecaster(
    forecaster            = forecaster,
    y                     = y,
    cv                    = cv,
    metric                = 'mean_absolute_error',
    n_jobs                = 'auto',
    verbose               = False,
    show_progress         = True
)
</code></pre>

</td>
</tr>
</table>
